<a href="https://colab.research.google.com/github/dhanushbabuk/HPE-CTY/blob/main/Fine_Tuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df= messages = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',delimiter='\t',names=['message','label'])

In [3]:
df.head()

,message,label
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [4]:
df.shape

(6920, 2)

In [5]:
#converting into dependent and indepentend features
X = list(df['message'])


In [6]:
y = list(df['label'])

In [7]:
# train test split
X_train , X_test,y_train ,y_test = train_test_split(X,y,test_size=0.20,random_state = 0)

In [8]:
X_train

['directed by kevin bray , whose crisp framing , edgy camera work , and wholesale ineptitude with acting , tone and pace very obviously mark him as a video helmer making his feature debut',
 'a deeply felt and vividly detailed story about newcomers in a strange new world',
 'for those for whom the name woody allen was once a guarantee of something fresh , sometimes funny , and usually genuinely worthwhile , hollywood ending is a depressing experience',
 "it 's one long bore",
 'all three women deliver remarkable performances',
 "if you saw it on tv , you 'd probably turn it off , convinced that you had already seen that movie",
 "the movie 's thesis elegant technology for the masses is surprisingly refreshing",
 "more concerned with overall feelings , broader ideas , and open ended questions than concrete story and definitive answers , soderbergh 's solaris is a gorgeous and deceptively minimalist cinematic tone poem",
 'very much a home video , and so devoid of artifice and purpose th

In [10]:
!pip install transformers

In [17]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

# Load tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [18]:
train_encodings = tokenizer(X_train,truncation=True,padding=True)
test_encodings = tokenizer(X_test ,truncation=True,padding=True)

In [19]:
train_encodings
# by default size is 512

{'input_ids': [[101, 2856, 2011, 4901, 19743, 1010, 3005, 15594, 20241, 1010, 3968, 6292, 4950, 2147, 1010, 1998, 17264, 1999, 23606, 18679, 2007, 3772, 1010, 4309, 1998, 6393, 2200, 5525, 2928, 2032, 2004, 1037, 2678, 16254, 2121, 2437, 2010, 3444, 2834, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1037, 6171, 2371, 1998, 14954, 2135, 6851, 2466, 2055, 24159, 1999, 1037, 4326, 2047, 2088, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2005, 2216, 2005, 3183, 1996, 2171, 13703, 5297, 2001, 2320, 1037, 11302, 1997, 2242, 4840, 1010, 2823, 6057, 1010, 1998, 2788, 15958, 4276, 19927, 1010, 5365, 4566, 2003, 1037, 2139, 24128, 3325, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2009, 1005, 1055, 2028, 2146, 8501, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
# converting encoding into dataset objects
import tensorflow as tf

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(len(X_train)).batch(8)  # Batch size 8, for shuffuling the training data

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16)  # Batch size 16 for evaluation


In [21]:
train_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 66), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 66), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [23]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

# Loaded tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Tokenizing training and testing data
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Creating TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(len(X_train)).batch(8)  # Batch size 8, shuffling the training data and also can be changed to 4

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16)  # Batch size 16 for the evaluation also can be changed to 8

# Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(train_dataset, epochs=2, validation_data=test_dataset)

# Evaluating the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/2


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
692/692 [==============================] - 119s 99ms/step - loss: 0.7152 - accuracy: 0.4812 - val_loss: 0.6931 - val_accuracy: 0.4668
Epoch 2/2
87/87 [==============================] - 4s 42ms/step - loss: 0.6931 - accuracy: 0.4668
Test Loss: 0.6931467056274414, Test Accuracy: 0.4667630195617676


## Unable to use these commands mentioned below it throws an error**

In [ ]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")



# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train_dataset,
#     eval_dataset=small_eval_dataset,
#     compute_metrics=compute_metrics,
# )



# trainer.train()